<h2>Анализ базы данных сервиса для чтения книг по подписке<a class="tocSkip"></h2

Компания-заказчик купила крупный сервис для чтения книг по подписке

Нам поставлена задача — проанализировать базу данных и помочь сформулировать ценностное предложение по новому продукту.

<h4>Задача:<a class="tocSkip"></h4

Проанализировать базу данных:

Посчитать, сколько книг вышло после 1 января 2000 года;

Для каждой книги посчитать количество обзоров и среднюю оценку;

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<h4>Входные данные:<a class="tocSkip"></h4

База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.



<h5>таблица books cодержит данные о книгах:<a class="tocSkip"></h5

book_id — идентификатор книги;

author_id — идентификатор автора;

title — название книги;

num_pages — количество страниц;

publication_date — дата публикации книги;

publisher_id — идентификатор издателя.

<h5>таблица authors cодержит данные об авторах:<a class="tocSkip"></h5

author_id — идентификатор автора;

author — имя автора

<h5>таблица publishers cодержит данные об издательствах:<a class="tocSkip"></h5

publisher_id — идентификатор издательства;

publisher — название издательств

<h5>таблица ratings cодержит данные о пользовательских оценках книг:<a class="tocSkip"></h5

rating_id — идентификатор оценки;

book_id — идентификатор книги;

username — имя пользователя, оставившего оценку;

rating — оценка книги.

<h5>таблица reviews cодержит данные о пользовательских обзорах на книги:<a class="tocSkip"></h5

review_id — идентификатор обзора;

book_id — идентификатор книги;

username — имя пользователя, написавшего обзор;

text— текст обзора.

<h2>Подключение к базе данных<a class="tocSkip"></h2

In [9]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [10]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<h3>Знакомимся с таблицами -  выводим первые строки<a class="tocSkip"></h3

In [11]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = f'''SELECT * FROM {table} LIMIT 5'''
    con=engine.connect()
    print(f'Таблица {table}')
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    print('=======================================')

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<h3>Посчитаем, сколько книг вышло после 1 января 2000 года<a class="tocSkip"></h3

In [12]:
query = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


<h3>Для каждой книги посчитаем количество обзоров и среднюю оценку<a class="tocSkip"></h3

In [13]:
query = '''
        SELECT title,
               COUNT(DISTINCT(review_id)) AS reviews_count,
               ROUND(AVG(rating),3) AS rating_avg
        FROM books AS b
        LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        ORDER BY reviews_count DESC, rating_avg DESC
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,reviews_count,rating_avg
0,Twilight (Twilight #1),7,3.663
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.288
3,The Book Thief,6,4.264
4,The Glass Castle,6,4.207
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000
996,Leonardo's Notebooks,0,4.000
997,Essential Tales and Poems,0,4.000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667


Всего в каталоге 1000 произведений.

Первая часть серии книг "Twilight " имеет 7 обзоров и является лидером по их количеству.
Остальные произведения сервиса имеют от 6 обзоров до их отсутствия.

Также по каждому произведению указан их средний рейтинг.
Средний рейтинг не корелирует с количеством обзоров  на книгу, скорее всего речь о личном вкусовом предпочтении читателя.

<h3>Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры<a class="tocSkip"></h3

In [14]:
query = '''
        WITH 
-- исключаем брошюры --
tab AS (
SELECT publisher_id,
       book_id
FROM books
WHERE num_pages > 50
)

SELECT tab.publisher_id,
       p.publisher
FROM tab 

INNER JOIN publishers AS p
ON tab.publisher_id = p.publisher_id

GROUP BY tab.publisher_id, p.publisher
ORDER BY COUNT(tab.book_id) DESC

LIMIT 1

        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,212,Penguin Books


Наиболее широко представлены произведения с размером более 50 страниц от издательства Penguin Books( британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам)

<h3>Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками<a class="tocSkip"></h3

In [15]:
query = '''
        SELECT author,
               ROUND(AVG(rating),3) AS rating_avg
        FROM books AS b
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        LEFT JOIN authors AS a ON b.author_id = a.author_id
        WHERE b.book_id IN (SELECT book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50)
        GROUP BY author
        ORDER BY rating_avg DESC
        LIMIT 1
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287


Самые высокий средний рейтинг книг - 4,29 баллов (по 5-балльной шкале) у Джоан Роулинг, автора книг во вселенной о Гарри Поттере.
Также в авторстве указан американский иллюстратор и писатель Мари Гранпрэ, которая подготовила иллюстрации к обложкам и главам книг.

<h3>Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.<a class="tocSkip"></h3

In [16]:
query = '''
        WITH reviews_count AS
        (SELECT username, COUNT(review_id) 
        FROM reviews
        WHERE username IN (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(rating_id) > 48)
        GROUP BY username)
        SELECT AVG(count) AS reviews_count_avg
        FROM reviews_count
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,reviews_count_avg
0,24.0


В среднем, пользователи, которые проявляют активность и ставят более 48 оценок, составляют обзоров на 24 книги.

<h3>Вывод<a class="tocSkip"></h3

819 книг вышло после 1 января 2000 года вышло

Больше всего количество обзоров у Twilight - 7

Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц

У J.K. Rowling/Mary GrandPré Самая высокая средняя оценка - 4.287

В среднем, пользователи, которые проявляют активность и ставят более 48 оценок, составляют обзоров на 24 книги.